In [1]:
import os
import glob
import random
import time

import numpy as np
from tqdm import tqdm

from fbp import FBPDataset
from fbp.src.utils import add_noise, delete_nans

In [ ]:
# Load npz files and metadata
npz_files = glob.glob("/scratch/gpi/seis/HIPER2/delsuc/Seisbench/PhaseNet_DAS_03_04_2025/data_npz/*")[:20]
metadata_path = "/scratch/gpi/seis/HIPER2/delsuc/Seisbench/PhaseNet_DAS_03_04_2025/metadata"

In [ ]:
# Create the dataset
dataset = FBPDataset(npz_files=npz_files,
                     metadata_path=metadata_path,
                     sigma=sigma,
                     shape=(1, 32, 2048),
                     overlap=0.9,
                     filter_kwargs=dict(type="bandpass",
                                        freqmin=2.5,
                                        freqmax=16))

In [ ]:
# Save dataset as single npz file
for i in tqdm(range(len(dataset))):
    # Event count for step function
    event_count = 0
    for idx in range(dataset[i][1].shape[1]):
        diff = np.diff(dataset[i][1][:, idx, :])
        if np.max(diff) >= 0.98:
            event_count += 1

    if event_count > 28:
        pathname = "/scratch/gpi/seis/HIPER2/delsuc/Seisbench/PhaseNet_DAS_03_04_2025/npz_32_2048_bp2_5_8"
        file_num = str(random.randint(0, int(time.time())))

        current_dataset = list(dataset[i])  # Transform tuple to list for item assignment

        # Add this part if you would like to add gaussian noise to 
        # Taking only chunks with average SNR >= 4 dB.
        noisy = False
        # snr_chunks = delete_nans(dataset.snr_chunk[i])
        # if np.average(snr_chunks) >= 4:
        #     current_dataset[0] = add_noise(dataset=dataset[i][0],
        #                                    scale=(0.1, 0.4))
        #     noisy = True

        if noisy is False:
            filename = os.path.join(pathname, file_num)
        else:
            filename = os.path.join(pathname, f"{file_num}_noise")

        np.savez(file=filename,
                 data=current_dataset)